In [5]:
import io, os
import pandas as pd
import numpy as np
from pathlib import Path
import openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning,module='openpyxl')

yourInput = ['TW IV', '2022']
#================================================================================================================================================
fname_branch_data = [
    #FNAME, BRANCH NAME
    ['semanggi', '01-JKT-Semanggi'],   ['bali ', '25-CNT-Bali'],             ['batam', '15-WST-Batam'],          ['ambon', '42-EAST-Ambon'], 
    ['klg', '02-JKT-KelapaGading'],    ['balikpapan', '26-CNT-Balikpapan'],  ['bengkulu', '16-WST-Bengkulu'],    ['goronta', '43-EAST-Gorontalo'],
    ['gmt', '04-JKT-GajahMadaTower'],  ['banjarmas', '27-CNT-Banjarmasin'],  ['jambi', '17-WST-Jambi'],          ['kendari', '44-EAST-Kendari'],
    ['___', '03-JKT-Kuningan'],        ['jember', '28-CNT-Jember'],          ['lampung', '18-WST-Lampung'],      ['makassar', '45-EAST-Makassar'],
    ['puri','05-JKT-LippoPuri'],       ['kediri', '29-CNT-Kediri'],          ['medan', '19-WST-Medan'],          ['manado', '46-EAST-Manado'],
    ['kemang', '06-JKT-Kemang'],       ['kudus', '30-CNT-Kudus'],            ['padang', '20-WST-Padang'],        ['palu', '47-EAST-Palu'],
    ['pluit', '07-JKT-Peluit'],        ['kupang', '31-CNT-Kupang'],          ['pekanbaru', '21-WST-Pekanbaru'],  ['ternate', '48-EAST-Ternate'],
    ['bogor', '08-JBD-Bogor'],         ['bajo', '32-CNT-LabuhanBajo'],       ['palembang', '22-WST-Palembang'],
    ['mm', '09-JBD-Tangerang'],        ['malang', '33-CNT-Malang'],          ['lubukl', '23-WST-Lubuklinggau'],
    ['___', '10-JBD-Serang'],          ['mataram', '34-CNT-Mataram'],        ['pangkal', '24-WST-Pangkalpinang'],
    ['bandung', '11-JBD-Bandung'],     ['palangkara', '35-CNT-Palangkara'],
    ['cirebon', '13-JBD-Cirebon'],     ['pontianak', '36-CNT-Pontianak'],
    ['coc', '12-JBD-Cikarang'],        ['samarinda', '37-CNT-Samarinda'],
    ['tasik', '14-Tasikmalaya'],       ['semarang', '38-CNT-Semarang'],
                                       ['solo', '39-CNT-Solo'],
                                       ['surabaya', '40-CNT-Surabaya'], 
                                       ['yogya', '41-CNT-Yogya'],
]

fname_branch = pd.DataFrame(fname_branch_data, columns=['fname','branch'])

#================================================================================================================================================

rcsa_dir = 'D:/_____/Monthly Report/Oprisk/CONTOH/1. Cabang/'
rcsa_comp = pd.DataFrame()
rcsa_row = pd.DataFrame()

path = rcsa_dir
for root, dirs, files in os.walk(path):
    for file in files:
        if((file.endswith(".xlsx")) & 
           (file.__contains__('RCSA')) & 
           ~(file.__contains__('$')) & 
           ~(file.lower().__contains__('summary')) & 
           ((root+file).lower().__contains__(yourInput[0].lower())) & 
           ((root+file).lower().__contains__(yourInput[1].lower()))):
            
            wb = openpyxl.load_workbook(root+'/'+file, data_only = True)
            sh1 = wb['RCSA']
            
            print('executing > ',root[len(path):], file)
            
            rcsa_row.loc[0,'Root'] = root[len(path):]
            rcsa_row.loc[0,'Fname'] = file
            rcsa_row.loc[0,'dtModified'] = time.ctime(os.path.getmtime(root+'/'+file))
            
            for i in range(len(fname_branch)):
                if((root+file).lower().__contains__(fname_branch.loc[i,'fname'].lower())):
                    rcsa_row.loc[0,'branch'] = fname_branch.loc[i,'branch']
                    break
                elif(~(root+file).lower().__contains__(fname_branch.loc[i,'fname'].lower()) & (i==len(fname_branch)-1)):
                    rcsa_row.loc[0,'branch'] = 'n/a'
                    
            rcsa_row.loc[0,'Type']  = sh1['A3'].value
            
            for i in range(29):
                if sh1['L'+str(9+i)].value == "COMPOSITE":
                    rcsa_row.loc[0,(i+1)] = pd.NA
                else:
                    rcsa_row.loc[0,(i+1)] = sh1['L'+str(9+i)].value
                
            rcsa_comp = pd.concat([rcsa_comp, rcsa_row], ignore_index = True)

rcsa_comp['key_1'] = rcsa_comp['branch'] + rcsa_comp['Type']
rcsa_comp = rcsa_comp.sort_values(by=['key_1','dtModified'], ascending = [True, True], ignore_index=True)
rcsa_comp = rcsa_comp.drop_duplicates(subset=['key_1'], keep = 'first')

fname_branch = fname_branch.sort_values(by = ['branch'])

rcsa_comp_teller = rcsa_comp[rcsa_comp['Type'] == 'Teller'].drop(columns = ['Root','Fname','key_1','dtModified','Type'])
rcsa_comp_teller = pd.merge(fname_branch['branch'], rcsa_comp_teller, on = 'branch', how = 'left')

rcsa_comp_cs = rcsa_comp[rcsa_comp['Type'] != 'Teller'].drop(columns = ['Root','Fname','key_1','dtModified','Type'])
rcsa_comp_cs = pd.merge(fname_branch['branch'], rcsa_comp_cs, on = 'branch', how = 'left')

# OUTPUT ---
writer = pd.ExcelWriter(path + 'RCSA Rekap Output - ' + yourInput[0] + ' ' + yourInput[1] + '.xlsx', engine = 'xlsxwriter')
rcsa_comp.to_excel(writer, sheet_name = 'RCSA Directory', index = False)
rcsa_comp_teller.T.to_excel(writer, sheet_name = 'RCSA Teller')
rcsa_comp_cs.iloc[:,0:23].T.to_excel(writer, sheet_name = 'RCSA CS')
writer.close()

RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\Bogor\RCSA TW IV\CS TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\Bogor\RCSA TW IV\Teller TOOLs RCSA NOBU - (Teller) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\COC TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\COC TOOLs RCSA NOBU - (Teller) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\GMT RCSA Teller GMT.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\GMT TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\Kemang TOOLs RCSA NOBU - (Teller) TW IV 2022.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\klg RCSA NOBU - (CS) TW IV 2022 revisi.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 2022 - REVISI\klg TOOLs RCSA NOBU - (Teller) TW IV 2022 revisi.xlsx
RCSA
executing >  1. Jabodetabek\RCSA TW IV 20

RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Lubuklinggau TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Lubuklinggau TOOLs RCSA NOBU - (Teller) TW IV 2022 - Rev 1.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Lubuklinggau TOOLs RCSA NOBU - (Teller) TW IV 2022.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Medan TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Medan TOOLs RCSA NOBU - (Teller) TW IV 2022 - Rev 1.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Medan TOOLs RCSA NOBU - (Teller) TW IV 2022.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Padang RCSA NOBU - (CS) TW IV 2022 - Rev 1.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Padang TOOLs RCSA NOBU - (CS) TW IV 2022.xlsx
RCSA
executing >  4. West Area\RCSA TW IV 2022 - West Area\Padang TOOLs RCSA NOBU - (Teller) TW IV 2022 - Rev 1